In [14]:
import os
import sys
import argparse
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.spatial.transform import Rotation

def time_alignment(ft, pose):    
    ft_time = ft[:, 0]
    pose_time = pose[:, 0]

    fx_interp = np.interp(pose_time, ft_time, ft[:, 1])
    fy_interp = np.interp(pose_time, ft_time, ft[:, 2])
    fz_interp = np.interp(pose_time, ft_time, ft[:, 3])
    tx_interp = np.interp(pose_time, ft_time, ft[:, 4])
    ty_interp = np.interp(pose_time, ft_time, ft[:, 5])
    tz_interp = np.interp(pose_time, ft_time, ft[:, 6])

    force_interp = np.stack([fx_interp, fy_interp, fz_interp]).T
    torque_interp = np.stack([tx_interp, ty_interp, tz_interp]).T
    return force_interp, torque_interp

def convert_rotation(pose):
    '''
    Compute theta_z in euler angle and 2D rotation matrix from pose quaternion
    '''
    r_ez, rmat = [], []
    for p in pose:
        euler = Rotation.from_quat(p[4:]).as_euler('zyx')
        theta_z = euler[0] + np.pi / 2
        cosz, sinz = np.cos(theta_z), np.sin(theta_z)
        r = np.array([[cosz, -sinz],
                        [sinz, cosz]])
        r_ez.append(theta_z)
        rmat.append(r)
    r_ez = np.stack(r_ez)
    rmat = np.stack(rmat)
    return r_ez, rmat

def rotate_force(force, rmat):
    '''
    Rotate the force into robot base frame
    '''
    rotated_fx, rotated_fy = [], []
    for f, r in zip(force, rmat):
        rotated_fx_fy = r.T @ np.array([f[0], f[1]])
        rotated_fx.append(rotated_fx_fy[0])
        rotated_fy.append(rotated_fx_fy[1])

    rotated_fx = np.stack(rotated_fx)
    rotated_fy = np.stack(rotated_fy)
    return rotated_fx, rotated_fy


In [15]:
# Directory to load force/torque and pose data
data_dir = "/home/jc/tmp/01-17/slide_1_17_16_8_7"

ft = np.loadtxt(os.path.join(data_dir, 'ft300s.txt'), delimiter=',')
pose = np.loadtxt(os.path.join(data_dir, 'pose.txt'), delimiter=',')
force_interp, torque_interp = time_alignment(ft, pose)
r_ez, rmat = convert_rotation(pose)
rotated_fx, rotated_fy = rotate_force(force_interp, rmat)

# Use relative time for plot
pose_time = pose[:, 0]
relative_time = pose_time - pose_time[0]

nstep = pose.shape[0]

In [16]:
fig = go.Figure(data=[go.Scatter(x=relative_time, y=rotated_fx, name='fx'),
                      go.Scatter(x=relative_time, y=rotated_fy, name='fy')
                     ]
               )
fig.show()

In [17]:
fig = go.Figure(data=go.Scatter(x=relative_time, y=torque_interp[:,2], name='fw'))
fig.show()

In [10]:
fig = go.Figure(data=[go.Scatter(x=relative_time, y=pose[:,1], name='px'),
                      go.Scatter(x=relative_time, y=pose[:,2], name='py')
                     ]
               )
fig.show()

## Second Action

In [18]:
# Directory to load force/torque and pose data
data_dir = "/home/jc/tmp/01-17/slide_1_17_17_26_27"

ft = np.loadtxt(os.path.join(data_dir, 'ft300s.txt'), delimiter=',')
pose = np.loadtxt(os.path.join(data_dir, 'pose.txt'), delimiter=',')
force_interp, torque_interp = time_alignment(ft, pose)
r_ez, rmat = convert_rotation(pose)
rotated_fx, rotated_fy = rotate_force(force_interp, rmat)

# Use relative time for plot
pose_time = pose[:, 0]
relative_time = pose_time - pose_time[0]

nstep = pose.shape[0]

In [19]:
fig = go.Figure(data=[go.Scatter(x=relative_time, y=rotated_fx, name='fx'),
                      go.Scatter(x=relative_time, y=rotated_fy, name='fy')
                     ]
               )
fig.show()

In [20]:
fig = go.Figure(data=go.Scatter(x=relative_time, y=torque_interp[:,2], name='fw'))
fig.show()

In [21]:
fig = go.Figure(data=[go.Scatter(x=relative_time, y=pose[:,1], name='px'),
                      go.Scatter(x=relative_time, y=pose[:,2], name='py')
                     ]
               )
fig.show()